# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa
* SpaCy Language Model

In [5]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U pip;
!{python} -m pip install tensorflow[tensorflow-addons]==2.1;
!{python} -m pip install rasa[spacy];

Requirement already up-to-date: pip in /home/debian/.conda/envs/rasa-new/lib/python3.8/site-packages (20.1.1)
ERROR: Could not find a version that satisfies the requirement tensorflow[tensorflow-addons]==2.1 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2)
ERROR: No matching distribution found for tensorflow[tensorflow-addons]==2.1
  Using cached rasa-1.10.2-py3-none-any.whl (510 kB)
  Using cached colorhash-1.0.2-py2.py3-none-any.whl (6.0 kB)
  Using cached networkx-2.4-py3-none-any.whl (1.6 MB)
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)
  Using cached pymongo-3.8.0.tar.gz (649 kB)
  Using cached PyJWT-1.7.1-py2.py3-none-any.whl (18 kB)
  Using cached tqdm-4.45.0-py2.py3-none-any.whl (60 kB)
  Using cached APScheduler-3.6.3-py2.py3-none-any.whl (58 kB)
  Using cached coloredlogs-10.0-py2.py3-none-any.whl (47 kB)
  Using cached sklearn_crfsuite-0.3.6-py2.py3-none-any.whl (12 kB)
  Using cached prompt_toolkit-2.0.10-py3-none-any.

In [12]:
!{python} -m spacy download es_core_news_md

/home/debian/.conda/envs/rasa-new/bin/python: No module named spacy


## Downloading the Spanish Language Model

In [ ]:
!{python} -m spacy link es_core_news_md es --force;

## Import the Installations

In [ ]:
import rasa


# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [ ]:
from rasa.training_data import load_data
from rasa.config import RasaNLUModelConfig
from rasa.model import Trainer
from rasa import config
import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = load_data("data/nlu/nlu-papaito.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config_simple.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

## Evaluating the NLU model on a random text

In [ ]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [ ]:
from rasa_nlu.test import run_evaluation

run_evaluation("data/nlu/nlu-papaito.md", model_directory)

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [ ]:

#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [ ]:
from IPython.display import Image
from rasa.agent import Agent
 
agent = Agent('domain-papaito.yml')
#agent.visualize("data/core/stories-papaito.md", "story_graph.png", max_history=2)
#Image(filename="story_graph.png")

## Training a Dialogue Model

In [ ]:
from rasa.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy, FormPolicy
from rasa.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain-papaito.yml',
              policies=[MemoizationPolicy(max_history=5),
                        KerasPolicy(validation_split=0.1,epochs=400),
                        FormPolicy(),
                        fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('data/core/stories-papaito.md')

agent.train(training_data)

agent.persist('models/dialogue')

# Talk to your Bot

In [ ]:
#Starting the Bot
from rasa.agent import Agent
from rasa.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('models/dialogue', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions/actions.py,target=/app/actions rasa/rasa-sdk

## Creating the voice synthesizer

In [ ]:
#!git clone https://github.com/NVIDIA/tacotron2.git

from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.layers import TacotronSTFT, STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.text import text_to_sequence
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.denoiser import Denoiser
import numpy as np
import torch

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

## Start chat

In [ ]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        voice, sr = synthesize(to_synth, "orador")
        sd.play(voice, sr)
        response_file.close()        